In [ ]:
import pandas as pd
import numpy as np

def process_rooms_number(x):
    '''Функция заполнения стобца количество комнат целыми числами'''
    if pd.isna(x):
        return 1
    
    if isinstance(x, int):
        return x
    
    if x.isdigit():
        return int(x)
   
    if x == 'Студия':
        return 1
    
    if x == 'Своб. планировка':
        return 1
    
    if x == '> 9':
        return 10

    return 1

def prediction(data, type_of_house_dict, district_dict, floor_dict, rooms_number_dict, w = [0.25, 0.25, 0.25, 0.25]):
    '''
    Функция получения предсказания цены по переданному датафрейму,
    полученным словарям для разных характеристик и весам 
    '''
    prediction = []
    
    for i in range(data.shape[0]):
        
        price_type = type_of_house_dict[data.type_of_house.iloc[i]] \
        if data.type_of_house.iloc[i] in type_of_house_dict.keys() else mean_sq_meter_price
        
        price_district = district_dict[data.district.iloc[i]] \
        if data.district.iloc[i] in district_dict.keys() else mean_sq_meter_price
        
        price_floor = floor_dict[data.floor.iloc[i]] \
        if data.floor.iloc[i] in floor_dict.keys() else mean_sq_meter_price
        
        price_room = rooms_number_dict[data.rooms_number.iloc[i]] \
        if data.rooms_number.iloc[i] in rooms_number_dict.keys() else mean_sq_meter_price
        
        prediction.append(round((price_type * w[0] + price_district * w[1] + price_floor * w[2] + \
                         price_room * w[3]) * data.area.iloc[i]))
        
    return prediction

In [ ]:
# Считаем данные
data_train = pd.read_csv('real_estate_novosibirsk.csv')
data_test = pd.read_table('dataset_521000_13.txt', delimiter=';')

# Удалим дубликаты и записи с пропусками
data_train = data_train.drop_duplicates(subset=['item_id'], keep='last')
data_train = data_train.dropna(subset=['area'])
data_train = data_train.dropna(subset=['type_of_house'])

# Заполним кол-во комнат целыми числами
data_train['rooms_number'] = data_train['rooms_number'].apply(process_rooms_number).copy()

# Удаляем 2% самых высоких и самых низких цен
data_train = data_train[(data_train.price.quantile(0.02) < data_train.price) & (data_train.price < data_train.price.quantile(0.99))]

# Ограничим все первыми 30 этажами
data_train = data_train[data_train.floor <= 30]

# Посчитаем среднюю цену среди всех квартир
mean_sq_meter_price = (data_train.price / data_train.area).mean()

# Создадим столбец с ценой за квадратный метр в каждой квартире
data_train['price_m2'] = data_train['price'] / data_train['area']

#Словари для учета специфики конкретной квартиры
# Создадим словарь тип дома: средняя цена
type_of_house_dict = data_train.groupby('type_of_house')['price_m2'].median().round().to_dict()

# Создадим словарь район: медианная цена
district_dict = data_train.groupby('district')['price_m2'].median().to_dict()

# Создадим словарь этаж: средняя цена
floor_dict = data_train.groupby('floor')['price_m2'].median().round().to_dict()

# Создадим словарь количество комнат: средняя цена
rooms_number_dict = data_train.groupby('rooms_number')['price_m2'].median().round()

In [ ]:
# Получим предсказания на трейне
pred = prediction(data_train, type_of_house_dict, district_dict, floor_dict, rooms_number_dict)

# Метрика на train
MAPE = round(((data_train.price - pred) / data_train.price).abs().mean(), 3)
MAPE

In [ ]:
# Получи
pred_test = prediction(data_test, type_of_house_dict, district_dict, floor_dict, rooms_number_dict)

answer = pd.Series(pred_test)
answer.to_csv('solution.csv', header=False, index=False)